In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('result/train/05_train_승인매출정보_KMeans.parquet')

df

,ID,기준년월,최종이용일자_기본,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_할부,이용건수_신용_B0M,이용건수_할부_B0M,이용건수_할부_유이자_B0M,...,이용개월수_D페이_R6M,이용금액_D페이_B0M,이용개월수_선결제_R6M,이용횟수_연체_R6M,가맹점매출금액_B1M,건수_할부전환_R6M,승인거절건수_R3M,승인거절건수_BL_R3M,승인거절건수_기타_R3M,Segment
0,TRAIN_000000,201807,20180719,20180719,10101,20180203,20180713,11,1,0,...,0,0,0,1,0,0,3,0,0,E
1,TRAIN_000001,201807,20180719,20170728,20170327,10101,20171231,13,0,0,...,0,0,0,0,0,0,3,0,0,B
2,TRAIN_000002,201807,20180706,20180706,20151119,20141230,20180627,12,0,0,...,0,0,0,0,6190,0,0,0,0,B
3,TRAIN_000003,201807,20180721,20180721,10101,20141111,20180715,6,1,1,...,0,0,0,0,0,0,3,0,0,E
4,TRAIN_000004,201807,20180124,10101,10101,20180512,10101,-2,0,0,...,0,0,0,0,0,0,0,0,0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399995,201812,20181220,10101,10101,20181212,20160501,2,0,0,...,0,0,0,0,0,0,0,0,0,C
2399996,TRAIN_399996,201812,20181202,10101,20170112,10101,20180112,10,0,0,...,0,0,0,0,0,0,0,0,0,A
2399997,TRAIN_399997,201812,20181230,10101,10101,20131124,20180919,10,0,0,...,0,0,0,0,0,0,0,0,0,C
2399998,TRAIN_399998,201812,20161224,10101,10101,10101,20150122,-2,0,0,...,0,0,0,0,0,0,0,0,0,A


In [3]:
# 제외할 컬럼 지정
exclude_cols = ['ID']

# 수치형 컬럼에서 ID는 제외
df_numeric = df.select_dtypes(include=[np.number]).drop(columns=exclude_cols, errors='ignore').copy()

# Segment_numeric 추가
df_numeric['Segment_numeric'] = df['Segment'].astype('category').cat.codes

# Segment_numeric과의 상관계수 계산
corr_with_segment = df_numeric.corr()['Segment_numeric'].drop('Segment_numeric')

# 상관계수 ≤ 0.2인 컬럼 제거
cols_to_drop = corr_with_segment[abs(corr_with_segment) <= 0.2].index.tolist()

# 원본 df에서 제거 (ID는 유지됨)
df_filtered = df.drop(columns=cols_to_drop)

# 결과 확인
print("제거된 컬럼 수:", len(cols_to_drop))
print("남은 컬럼 수:", df_filtered.shape[1])

제거된 컬럼 수: 66
남은 컬럼 수: 7


In [4]:
df_filtered

,ID,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_할부,이용후경과월_할부,Segment
0,TRAIN_000000,20180719,10101,20180203,20180713,2,E
1,TRAIN_000001,20170728,20170327,10101,20171231,9,B
2,TRAIN_000002,20180706,20151119,20141230,20180627,0,B
3,TRAIN_000003,20180721,10101,20141111,20180715,1,E
4,TRAIN_000004,10101,10101,20180512,10101,12,D
...,...,...,...,...,...,...,...
2399995,TRAIN_399995,10101,10101,20181212,20160501,12,C
2399996,TRAIN_399996,10101,20170112,10101,20180112,9,A
2399997,TRAIN_399997,10101,10101,20131124,20180919,1,C
2399998,TRAIN_399998,10101,10101,10101,20150122,12,A


In [5]:
df_filtered.to_parquet('result/train/06_train_승인매출정보_세그먼트상관계수.parquet', index=False)
df_filtered.to_csv('result/train/06_train_승인매출정보_세그먼트상관계수.csv', index=False)